In [1]:
import numpy as np
import pandas as pd

import ast

In [2]:
data = pd.read_csv('scr/Datasets/movies_dataset_ETL_toML.csv', usecols=['id', 'title', 'overview', 'genres'])

In [3]:
data.head()

,id,title,overview,genres
0,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","['Animation', 'Comedy', 'Family']"
1,8844.0,Jumanji,When siblings Judy and Peter discover an encha...,"['Adventure', 'Fantasy', 'Family']"
2,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...,"['Romance', 'Comedy']"
3,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...,['Comedy']
4,949.0,Heat,"Obsessive master thief, Neil McCauley leads a ...","['Action', 'Crime', 'Drama', 'Thriller']"


In [4]:
data['tags'] = data['overview'].astype(str) + ". " + data['genres'].astype(str)
data.drop(columns=['overview', 'genres'])

,id,title,tags
0,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844.0,Jumanji,When siblings Judy and Peter discover an encha...
2,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...
4,949.0,Heat,"Obsessive master thief, Neil McCauley leads a ..."
...,...,...,...
4995,79927.0,The Pope Must Die,A less-than-qualified and far-from-perfect pri...
4996,460135.0,LEGO DC Super Hero Girls: Brain Drain,"When Supergirl, Wonder Woman, Batgirl, Bumbleb..."
4997,426272.0,Take Me,Ray is a fledgling entrepreneur who specialize...
4998,434873.0,It Stains the Sands Red,"In the throes of a zombie apocalypse, A troubl..."


In [5]:
data['tags'] = data['tags'].str.lower()

El siguiente bloque de código elimina todo caracter que no sea de la 'a' a la 'z'.

In [6]:
import re
data["tags"] = data["tags"].apply(lambda x: re.sub("[^a-zA-Z]"," ",str(x)))

para reducir el numero de palabras en los datos se puede utilizar la siguiente librería, para que la morfología de las palabras no afecte en la frecuencia de éstas.

In [7]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [8]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [9]:
data["tags"] = data["tags"].apply(stem)

Modelo

La similitud del coseno se utiliza para encontrar cuantas palabras se repiten entre los diferentes registros. Así que para utilizarlo se deben extraer la cantidad de veces que se repiten ñas palabras. en este caso: las 1500 palabras más comunes.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000, stop_words="english")

In [11]:
vectors = cv.fit_transform(data["tags"]).toarray()
vectors.shape

(5000, 1000)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
similares = cosine_similarity(vectors)
pd.DataFrame(similares).shape

(5000, 5000)

In [13]:
def recomendacion(movie):
    movie = str.lower(movie)
    #busca el index del titulo en minusculas
    movie_index = data[data["title"].apply(str.lower) == movie].index[0]

    distances = similares[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:6]
    for i in movie_list:
        print(data.iloc[i[0]].title)

In [14]:
recomendacion("Jumanji")

Indie Game: The Movie
Wreck-It Ralph
Brainscan
The Spiderwick Chronicles
Video Games: The Movie


In [15]:
recomendacion("Toy Story")

Toy Story 2
The 40 Year Old Virgin
For Love or Money
Toy Story 3
In a Heartbeat
